In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import unet
from malaya_speech.utils import tf_featurization
import IPython.display as ipd
import numpy as np

In [4]:
class Model:
    def __init__(self, size = 2):
        self.X = tf.placeholder(tf.float32, (None))
        self.Y = tf.placeholder(tf.float32, (2, None))
        
        stft_X, D_X = tf_featurization.get_stft(self.X)
        
        self.stft = []
        for i in range(size):
            self.stft.append(tf_featurization.get_stft(self.Y[i]))
        
        self.outputs = []
        for i in range(size):
            with tf.variable_scope(f'model_{i}'):
                self.outputs.append(unet.model.apply_unet(D_X))
        
        self.loss = []
        for i in range(size):
            self.loss.append(tf.reduce_mean(tf.abs(self.outputs[i] - self.stft[i][1])))        
        
        self.cost = tf.reduce_sum(self.loss)
        
        separation_exponent = 2
        EPSILON = 1e-10
        
        output_sum = tf.reduce_sum([o ** separation_exponent for o in self.outputs], axis=0) + EPSILON
        
        self.istft = []
        for no, D in enumerate(self.outputs):

            instrument_mask = (D ** separation_exponent + (EPSILON / size)) / output_sum
            instrument_mask = tf_featurization.extend_mask(instrument_mask)
            old_shape = tf.shape(instrument_mask)
            new_shape = tf.concat(
                [[old_shape[0] * old_shape[1]], old_shape[2:]],
                axis=0)
            instrument_mask = tf.reshape(instrument_mask, new_shape)

            instrument_mask = instrument_mask[:tf.shape(stft_X)[0]]
            masked_stft = tf.cast(instrument_mask, dtype=tf.complex64) * stft_X
            self.istft.append(tf_featurization.istft(masked_stft, self.X)[:,0])

In [5]:
tf.compat.v1.reset_default_graph()
model = Model()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 44100)
len(y) / sr

5.630680272108844

In [7]:
ipd.Audio(y, rate = sr)

In [8]:
y_ = malaya_speech.augmentation.waveform.sox_augment_high(y, min_bass_gain = 70,
                                                          reverberance = 50, 
                                                          negate = 0)
ipd.Audio(y_, rate = sr)

In [9]:
noise = y - y_

In [10]:
stft = sess.run(model.stft, feed_dict = {model.Y: [y, noise]})

In [11]:
stft[0][1].shape

(1, 512, 1024, 1)

In [12]:
outputs = sess.run(model.outputs, feed_dict = {model.X: y_})
[o.shape for o in outputs]

[(1, 512, 1024, 1), (1, 512, 1024, 1)]

In [13]:
sess.run(model.loss, feed_dict = {model.X: y_, model.Y: [y, noise]})

[0.21449338, 0.19405052]

In [14]:
istft = sess.run(model.istft, feed_dict = {model.X: y_})
[s.shape for s in istft]

[(248313,), (248313,)]

In [15]:
ipd.Audio(istft[0], rate = sr)

In [16]:
ipd.Audio(istft[1], rate = sr)

In [17]:
ipd.Audio(y_, rate = sr)